<a href="https://colab.research.google.com/github/SammyDMartin/CC/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Uncomment for Colab
use_cc = False
"""
!pip install "ray[rllib]" torch
!pip install nashpy
!pip install pickle5
!git clone https://github.com/SammyDMartin/CC.git
!cd CC

use_CC = True
"""
import numpy as np

In [2]:
if use_cc == True:
    from CC.cc_search import assess_agent_vs_others,assess_agent_vs_agent,make_selfplay_checkpoint
    from CC.cc_tools import DQNTorchPolicy,DQNTrainer,PPOTorchPolicy,PPOTrainer,A3CTorchPolicy,A3CTrainer,CoopPolicy,DefectPolicy,RandPolicy,UncoopExploiterPolicy, BullyPolicy, MaximinPolicy,MaximaxPolicy,MindlessCoopPolicy,MindlessCoopSecurityPolicy
    from CC.cc_tools import IteratedPrisonersDilemma, IteratedChicken,IteratedStagHunt, IteratedBoS, MutalismCoordination, PureCoordination,IteratedAsymBoS
else:
    from cc_search import assess_agent_vs_others,assess_agent_vs_agent,make_selfplay_checkpoint
    from cc_tools import DQNTorchPolicy,DQNTrainer,PPOTorchPolicy,PPOTrainer,A3CTorchPolicy,A3CTrainer,CoopPolicy,DefectPolicy,RandPolicy,UncoopExploiterPolicy, BullyPolicy, MaximinPolicy,MaximaxPolicy,MindlessCoopPolicy,MindlessCoopSecurityPolicy
    from cc_tools import IteratedPrisonersDilemma, IteratedChicken,IteratedStagHunt, IteratedBoS, MutalismCoordination, PureCoordination,IteratedAsymBoS

ray version: 1.9.2


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from cc_tools import bully, security_vals,maximin,nash_from_RLLIB,maximax,mindless_cooperate
from cc_tools import IteratedBoS, IteratedPrisonersDilemma

# Tests that all matrix games are working

In [5]:
games = [IteratedPrisonersDilemma, IteratedChicken,IteratedStagHunt, IteratedBoS, MutalismCoordination, PureCoordination,IteratedAsymBoS]

def pretty_print_matrix(g):
    m_list = g.PAYOUT_MATRIX.tolist()
    print(m_list[0])
    print(m_list[1])


def test_matrix(g):
    print(g.__name__)
    pretty_print_matrix(g)
    #print(nash_from_RLLIB(g))
    print("\nBully action steps (out of pure and 50/50 strats): {}".format(bully(g)))
    print("Maximin action steps (out of pure and 50/50 strats): {}".format(maximin(g)))
    print("Maximax action steps (out of pure and 50/50 strats): {}".format(maximax(g)))
    print("Mindless Coop action steps (out of pure and 50/50 strats): {}".format(mindless_cooperate(g)))
    print("Security Vals: {}".format(security_vals(g)))
    print("\n\n")

for g in games:
    test_matrix(g)

IteratedPrisonersDilemma
[[-1, -1], [-3, 0]]
[[0, -3], [-2, -2]]

Bully action steps (out of pure and 50/50 strats): (array([0., 1.]), array([0., 1.]))
Maximin action steps (out of pure and 50/50 strats): (array([0., 1.]), array([0., 1.]))
Maximax action steps (out of pure and 50/50 strats): (array([0., 1.]), array([0., 1.]))
Mindless Coop action steps (out of pure and 50/50 strats): (array([1., 0.]), array([1., 0.]))
Security Vals: (-2.0, -2.0)



IteratedChicken
[[0.0, 0.0], [-1.0, 1.0]]
[[1.0, -1.0], [-10.0, -10.0]]

Bully action steps (out of pure and 50/50 strats): (array([0., 1.]), array([0., 1.]))
Maximin action steps (out of pure and 50/50 strats): (array([1., 0.]), array([1., 0.]))
Maximax action steps (out of pure and 50/50 strats): (array([0., 1.]), array([0., 1.]))
Mindless Coop action steps (out of pure and 50/50 strats): (array([0.5, 0.5]), array([0.5, 0.5]))
Security Vals: (-1.0, -1.0)



IteratedStagHunt
[[3, 3], [0, 2]]
[[2, 0], [1, 1]]

Bully action steps (out of pure

# Tests single Uncooperative agent vs DQN

In [6]:
DQN_c,c_res = make_selfplay_checkpoint(IteratedBoS,(DQNTorchPolicy,DQNTrainer),20)

2022-04-20 12:04:50,017	ERROR syncer.py:111 -- Log sync requires rsync to be installed.
 pid=1632) 2022-04-20 12:04:54,218	INFO dqn.py:142 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
 pid=1632) 2022-04-20 12:04:54,218	INFO trainer.py:745 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
 pid=1632) 2022-04-20 12:04:54,236	WARNING deprecation.py:46 -- DeprecationWarning: `convert_to_non_torch_type` has been deprecated. Use `ray/rllib/utils/numpy.py::convert_to_numpy` instead. This will raise an error in the future!
 pid=1632) 2022-04-20 12:04:54,261	WARNING util.py:57 -- Install gputil for GPU system monitoring.
 pid=1632) 2022-04-20 12:04:54,273	WARNING deprecation.py:46 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` inst

In [7]:
unrol_df = lambda df : df.trial_dataframes[list(df.trial_dataframes.keys())[0]]

In [8]:
unrol_df(c_res)

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_healthy_workers,timesteps_total,timesteps_this_iter,agent_timesteps_total,done,...,info/learner/player_col/learner_stats/mean_q,info/learner/player_col/learner_stats/min_q,info/learner/player_col/learner_stats/max_q,info/learner/player_col/learner_stats/cur_lr,info/learner/player_row/learner_stats/allreduce_latency,info/learner/player_row/learner_stats/grad_gnorm,info/learner/player_row/learner_stats/mean_q,info/learner/player_row/learner_stats/min_q,info/learner/player_row/learner_stats/max_q,info/learner/player_row/learner_stats/cur_lr
0,35.0,10.0,21.000000,10.0,10,0,100,0,200,False,...,-0.159976,-0.800350,0.231392,0.0001,0.0,0.137531,0.166969,-0.111159,0.611491,0.0001
1,40.0,0.0,21.666667,10.0,20,0,300,0,600,False,...,0.067854,-0.428155,0.369486,0.0001,0.0,0.154271,0.465442,0.005348,0.830526,0.0001
2,40.0,0.0,23.100000,10.0,20,0,500,0,1000,False,...,0.203865,-0.239065,0.559817,0.0001,0.0,0.157205,0.636698,0.094358,1.195363,0.0001
3,40.0,0.0,22.428571,10.0,20,0,700,0,1400,False,...,0.472591,-0.005899,0.771223,0.0001,0.0,0.156483,0.927071,0.219262,1.615749,0.0001
4,45.0,0.0,23.611111,10.0,20,0,900,0,1800,False,...,0.647888,0.243151,0.960108,0.0001,0.0,0.160702,1.262588,0.378902,2.066742,0.0001
5,45.0,0.0,24.450000,10.0,20,0,1100,0,2200,False,...,1.027328,0.546021,1.396501,0.0001,0.0,0.251530,1.647541,0.608889,2.651393,0.0001
6,50.0,5.0,25.200000,10.0,20,0,1300,0,2600,False,...,1.421226,0.883671,1.844686,0.0001,0.0,0.123586,2.396735,0.916011,3.257569,0.0001
7,50.0,10.0,25.250000,10.0,20,0,1500,0,3000,False,...,1.640742,1.265978,2.388669,0.0001,0.0,0.220725,2.844042,1.246486,3.929536,0.0001
8,50.0,10.0,26.000000,10.0,20,0,1700,0,3400,False,...,2.206614,1.647885,2.985011,0.0001,0.0,0.328254,2.894952,1.599483,4.630910,0.0001
9,50.0,10.0,25.850000,10.0,20,0,1900,0,3800,False,...,2.631813,2.128001,3.717715,0.0001,0.0,0.232559,4.266098,1.948761,5.467805,0.0001


In [9]:
exanal = assess_agent_vs_agent(policies=[DQNTorchPolicy,UncoopExploiterPolicy],checkpoints=[DQN_c],environment=IteratedBoS,iters=5)
unrol_df(exanal)

 pid=3756) ray version: 1.9.2
 pid=3756) going to load policy player_row from checkpoint C:\Users\User\ray_results\test\DQN_IteratedBoS_b25e4_00000_0_2022-04-20_12-04-49\checkpoint_000020\checkpoint-20


 pid=3756) 2022-04-20 12:05:35,802	INFO dqn.py:142 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
 pid=3756) 2022-04-20 12:05:35,802	INFO trainer.py:745 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
 pid=3756) 2022-04-20 12:05:35,824	WARNING deprecation.py:46 -- DeprecationWarning: `convert_to_non_torch_type` has been deprecated. Use `ray/rllib/utils/numpy.py::convert_to_numpy` instead. This will raise an error in the future!
 pid=3756) restoring ckpt: not loading objs['filters']
 pid=3756) 2022-04-20 12:05:35,864	WARNING util.py:57 -- Install gputil for GPU system monitoring.
 pid=3756) 2022-04-20 12:05:35,875	WARNING deprecation.py:46 -- DeprecationWarning: `policy_mapping_fn(agent_id)` has been deprecated. Use `policy_mapping_fn(agent_id, episode, worker, **kwargs)` instead. This will raise an error in 

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_healthy_workers,timesteps_total,timesteps_this_iter,agent_timesteps_total,done,...,timers/learn_time_ms,timers/learn_throughput,info/num_steps_sampled,info/num_agent_steps_sampled,info/num_steps_trained,info/num_agent_steps_trained,info/last_target_update_ts,info/num_target_updates,perf/cpu_util_percent,perf/ram_util_percent
0,0.0,0.0,0.0,10.0,100,0,1000,0,2000,False,...,0.0,0.0,1000,2000,32,64,1000,1,36.500000,80.614286
1,0.0,0.0,0.0,10.0,100,0,2000,0,4000,False,...,0.0,0.0,2000,4000,8032,16064,1504,2,35.425000,79.912500
2,0.0,0.0,0.0,10.0,100,0,3000,0,6000,False,...,0.0,0.0,3000,6000,16032,32064,2512,4,36.587500,79.175000
3,0.0,0.0,0.0,10.0,100,0,4000,0,8000,False,...,0.0,0.0,4000,8000,24032,48064,3520,6,38.985714,78.900000
4,0.0,0.0,0.0,10.0,100,0,5000,0,10000,True,...,0.0,0.0,5000,10000,32032,64064,4528,8,39.612500,78.950000


# Tests DQN vs Uncoop Agent on all games

In [ ]:
strategies = [UncoopExploiterPolicy,MindlessCoopPolicy,MaximinPolicy,MaximaxPolicy,RandPolicy,BullyPolicy,MindlessCoopSecurityPolicy]
#opponents = [(PPOTorchPolicy,PPOTrainer),(DQNTorchPolicy,DQNTrainer),(A3CTorchPolicy,A3CTrainer)]

opponents = [(DQNTorchPolicy,DQNTrainer),(PPOTorchPolicy,PPOTrainer)]

results_uncoop = assess_agent_vs_others((UncoopExploiterPolicy,None),opponents=opponents,strategies=strategies,steps=6)
results_uncoop[0]

In [ ]:
results_PPO = assess_agent_vs_others((PPOTorchPolicy,PPOTrainer),opponents=opponents,strategies=strategies,steps=6)
results_PPO[0]

In [12]:
results_uncoop[0]

,IteratedPrisonersDilemma,IteratedChicken,IteratedStagHunt,IteratedBoS,MutalismCoordination,PureCoordination,IteratedAsymBoS
0,-0.222222,-0.1,0.111111,0.123889,0.111111,0.058667,0.112444
1,-0.222222,-0.1,0.111111,0.125000,0.111111,0.058778,0.117556
2,-0.222222,-0.1,0.111111,0.120000,0.111111,0.051000,0.106000
3,-0.222222,-0.1,0.111111,0.127667,0.111111,0.057111,0.114222
4,-0.222222,-0.1,0.111111,0.122222,0.111111,0.056333,0.108667
5,-0.222222,-0.1,0.111111,0.127444,0.111111,0.057222,0.119778


In [13]:
results_PPO[0]

NameError: name 'results_PPO' is not defined

## Normalized results
(note that the DQN did worse than the baseline on everything except for Chicken and Mutualism)

In [14]:
results_PPO[0]-results_uncoop[0]

NameError: name 'results_PPO' is not defined